# AI Expariments

In [1]:
import tiktoken
from langchain_openai import OpenAIEmbeddings
import numpy as np
import bs4
from langchain_community.document_loaders import WebBaseLoader

import os
from dotenv import load_dotenv

load_dotenv()

os.environ['LANGCHAIN_TRACING_V2'] = 'true'
os.environ['LANGCHAIN_ENDPOINT'] = 'https://api.smith.langchain.com'

langchain_api_key = os.getenv("LANGCHAIN_API_KEY")
os.environ['LANGCHAIN_API_KEY'] = langchain_api_key

open_api_key = os.getenv("OPEN_API_KEY")

os.environ['OPENAI_API_KEY'] = open_api_key
os.environ['USER_AGENT'] = os.getenv("USER_AGENT")
USER_AGENT = os.getenv("USER_AGENT")

c:\Users\vmelnyk2\AppData\Local\Programs\Python\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


**Url parser**

In [2]:
import requests
from bs4 import BeautifulSoup

parsed = set()
links = ["https://mathmod.chnu.edu.ua/"]

start_url = "https://mathmod.chnu.edu.ua/"
depth = 10

for i in range(depth):
    new_links = []
    for link in links:
        if link not in parsed:
            try:
                response = requests.get(link, headers={"User-Agent": USER_AGENT})
                soup = BeautifulSoup(response.content, "html.parser")
                for a in soup.find_all("a", href=True):
                    href = a["href"]
                    if href.startswith("/"):
                        href = start_url + href[1:]
                    if href.startswith(start_url) and href not in parsed and 'pdf' not in href.lower() and 'jpg' not in href.lower() and 'png' not in href.lower() and 'jpeg' not in href.lower() and 'docx' not in href.lower() and 'doc' not in href.lower() and 'xls' not in href.lower() and 'xlsx' not in href.lower() and 'pptx' not in href.lower() and 'ppt' not in href.lower() and 'email-protection' not in href.lower() and 'mailto' not in href.lower() and 'tel' not in href.lower() and 'javascript' not in href.lower() and 'webp' not in href.lower() and 'svg' not in href.lower() and 'mp4' not in href.lower() and 'avi' not in href.lower() and 'mov' not in href.lower() and 'mkv' not in href.lower() and 'flv' not in href.lower() and 'wmv' not in href.lower() and 'mp3' not in href.lower() and 'wav' not in href.lower():
                        new_links.append(href)
            except Exception as e:
                print(f"Error fetching {link}: {e}")
            parsed.add(link)
    links.extend(new_links)
    links = list(set(links))  # remove duplicates
    i += 1

with open("links.txt", "w") as f:
    for line in links:
        f.write(line + "\n")

Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.


In [ ]:
import bs4
from langchain_community.document_loaders import WebBaseLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS

with open('links.txt', 'r') as file:
    # data = file.read().replace('\n', ',')
    data = file.read()
    
links = data.split('\n')

loader = WebBaseLoader(
    web_paths=links
)
blog_docs = loader.load()

text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
    chunk_size=300, 
    chunk_overlap=50)

# Make splits
splits = text_splitter.split_documents(blog_docs)
print(splits[:2])

vectorstore = FAISS.from_documents(splits, OpenAIEmbeddings())

vectorstore.save_local("../models/")

ModuleNotFoundError: No module named 'langchain.vectorstores'

In [ ]:
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings
from langchain_openai import ChatOpenAI
# from langchain.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
import pickle
from langchain_core.prompts import PromptTemplate, ChatPromptTemplate

embd = OpenAIEmbeddings()

save_path = "../models/"
# with open(f"../models/index.pkl", "rb") as f:
#     stored_data = pickle.load(f)

vectorstore_init = FAISS.load_local(folder_path="../models/", embeddings=embd, allow_dangerous_deserialization=True)

# Load all FAISS shards and merge them
faiss_indexes = [vectorstore_init]
for filename in os.listdir(save_path):
    if filename.startswith("faiss_index_"):  # Load only the FAISS shards
        index_path = os.path.join(save_path, filename)
        vectorstore = FAISS.load_local(index_path, embd, allow_dangerous_deserialization=True)
        faiss_indexes.append(vectorstore)

# Merge all FAISS indexes into a single one
if faiss_indexes:
    merged_vectorstore = faiss_indexes[0]
    for store in faiss_indexes[1:]:
        merged_vectorstore.merge_from(store)

# retriever = vectorstore.as_retriever(search_kwargs={"k": 5}) # number of documents retrieved

retriever = merged_vectorstore.as_retriever(search_kwargs={"k": 5})

# Prompt
template = """Answer the question based only on the following context:
{context}

Please, provide as much information as possible!
Do not say words like 'context'
Question: {question}
"""

prompt = ChatPromptTemplate.from_template(template)

# LLM
llm = ChatOpenAI(model_name="gpt-4o", temperature=0)

rag_chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

In [12]:
rag_chain.invoke("Хто такий Ігор Михайлович Черевко?")

Failed to send compressed multipart ingest: langsmith.utils.LangSmithUserError: This API key is org-scoped and requires workspace specification. Please provide 'workspace_id' parameter, or set LANGSMITH_WORKSPACE_ID environment variable.


'Ігор Михайлович Черевко — це доктор фізико-математичних наук, професор, завідувач кафедри математичного моделювання. Він розпочав свою трудову діяльність після закінчення Чернівецького державного університету у 1978 році на посаді асистента. З 1986 року працював старшим викладачем, доцентом кафедри прикладної математики, а згодом став завідувачем кафедри математичного моделювання. Ігор Михайлович також є науковим керівником студентських наукових робіт. Він має SCOPUS Author ID: 15520902400, Web of Science Researcher ID: G-3796-2017, ORCID ID: 0000-0002-2690-2091, та профіль у Google Scholar. За свої заслуги отримав Почесну відзнаку Чернівецької обласної ради «За заслуги перед Буковиною».'

Failed to send compressed multipart ingest: langsmith.utils.LangSmithUserError: This API key is org-scoped and requires workspace specification. Please provide 'workspace_id' parameter, or set LANGSMITH_WORKSPACE_ID environment variable.
